# NRC-23 - Image Quality Verification by Filter   

## Notebook: Create Synthetic PSF with WebbPSF

**Author**: Matteo Correnti, STScI Scientist II
<br>
**Created**: November, 2021
<br>
**Last Updated**: February, 2022

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
3. [Create a synthetic PSF model (with WebbPSF)](#webbpsf_intro)<br>
    3.1 [Create PSFs for all detectors and filters](#psfs)<br>
    3.2 [Create PSFs for a subset of filters](#psf_filt)<br>
    3.3 [Create PSFs for a subset of detectors](#psf_det)<br>
    3.4 [Create PSFs for a subset of detectors and filters](#psf_det_filt)<br>
    3.5 [Create PSFs for a subset of detectors and filters at specific positions in the detector](#psf_det_filt_pos)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

This notebook allows to create a library of synthetic PSFs for all the filters and detectors, needed for the analysis of NRC-23 (i.e., comparison between empirical PSFs and synthetic PSFs and PSFs model for PSF photometry). It is also possible to create single synthetic PSFs both located at the center of the detector (default) or at a (X,Y) position defined by the user.

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all the necessary Python packages and we define some plotting parameters.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import os 

from datetime import datetime

os.environ['WEBBPSF_PATH'] = '/Users/correnti/data/webbpsf-data'
os.environ['PYSIN_CDBS'] = '/grp/hst/cdbs/'

import webbpsf
from webbpsf.utils import to_griddedpsfmodel

import numpy as np

import pysynphot  # PYSIN_CDBS must be defined in the user's environment (see note below)

<div class="alert alert-block alert-warning">
    <h3><u><b>Warning</b></u></h3>

**Note on WebbPSF**: WebbPSF (along all its dependencies and required reference data) can be installed via `astroconda`. If you install with pip, you must manually download and install the data files, as [described](https://webbpsf.readthedocs.io/en/latest/installation.html) here.    
<div >

<div class="alert alert-block alert-warning">
    <h3><u><b>Warning</b></u></h3>

**Note on pysynphot**: Data files for pysynphot are distributed separately by Calibration Reference Data System. They are expected to follow a certain directory structure under the root directory, identified by the PYSYN_CDBS environment variable that must be set prior to using this package. In the example below, the root directory is arbitrarily named /my/local/dir/trds/. \
export PYSYN_CDBS=/my/local/dir/trds/ \
See documentation [here](https://pysynphot.readthedocs.io/en/latest/#installation-and-setup) for the configuration and download of the data files.    
<div >

3.<font color='white'>-</font>Create synthetic PSF (with WebbPSF) <a class="anchor" id="webbpsf_intro"></a>
------------------

WebbPSF is a Python package that computes simulated PSFs for NASA’s JWST and Nancy Grace Roman Space Telescope (formerly WFIRST). WebbPSF transforms models of telescope and instrument optical state into PSFs, taking into account detector pixel scales, rotations, filter profiles, and point source spectra. It is not a full optical model of JWST, but rather a tool for transforming optical path difference (OPD) maps, created with some other tool, into the resulting PSFs as observed with JWST’s or Roman’s instruments. For a full documentation on WebbPSF, see [here](https://webbpsf.readthedocs.io/en/latest/) and for is capability and limitation, see [here](https://webbpsf.readthedocs.io/en/latest/intro.html).

The function below allows to create a single PSF or a grid of PSFs (the PSF can also be saved as a fits file). First, we need to specify the instrument (NIRCam), detector, and filter. Then, to create a single (or grid) PSF we use the webbPSF method *psf_grid*, which will output a (list of or single) photutils GriddedPSFModel object(s). A tutorial notebook on the *psf_grid* method can be found [here](https://github.com/spacetelescope/webbpsf/blob/stable/notebooks/Gridded_PSF_Library.ipynb).

**Important Parameters**:

* `num`: The total number of fiducial PSFs to be created and saved in the files. This
    number must be a square number (4, 9, 16, etc.) or equal to 1 for a single PSF. 

* `oversample`: the oversample factor we want to adopt in the PSF creation.

* `fov`: the size in pixel of the PSF model. The size depends on the shape of the PSF and how much flux is contained in the wings of the PSFs (i.e., a small field of view will exclude more flux from the PSF wings). However, increasing the field of view, increase also the computational time, so we need to find a reasonable compromise.

* `all_detectors`: run all detectors for the instrument. If we want to create a PSF for a single detector or a list of detector, we need to to set `all_detectors = False` and define the detector instance (i.e., inst.detector = str, or loop over a list).

* `use_detsampled_psf`: If the grid of PSFs returned will be detector sampled (made by binning down the oversampled PSF) or oversampled by the factor defined by the oversample. For our analysis, we want to create an oversampled PSF model, so we set `use_detsampled_psf = False`.

* `add_distortion`: if True will give you the distorted PSF (similar to cal.fits - level2 images), otherwise it will provide the undistorted PSF (similar to i2d.fits - level3 images).

The main parameters that we need to modify to create the two set of PSF libraries (i.e., comparison with empirical PSFs and PSF photometry) are the following: `fov` and `num`. 

For the PSF photometry, we can use a small field of view, since the fit is peformed only in the central region of the PSF (usually in a ~ 5 px size box), whereas when we do the comparison with the empirical PSFs we want to use a larger field of view to include also the wings. For what concerns the numbers of PSFs in the grid, it depends on how many good PFS stars we have in the field of view, and thus how many empirical PSFs we can build (and also if we just want to do a quick comparison with a single PSF, without taking into account the second-order field-dependent effects). For the PSF photometry, we can create a more finer grid (i.e., num = 16 or 25). 

<div class="alert alert-block alert-info">
    
**Note on centering**: by default, the PSF will be centered at the exact center of the output array. This means that if the PSF is computed on an array with an odd number of pixels, the PSF will be centered exactly on the central pixel. If the PSF is computed on an array with even size, it will be centered on the “crosshairs” at the intersection of the central four pixels.

**Note on normalization**: by default, PSFs are normalized to total intensity = 1.0 at the entrance pupil (i.e. at the JWST OTE primary). A PSF calculated for an infinite aperture would thus have integrated intensity =1.0. A PSF calculated on any smaller finite subarray will have some finite encircled energy less than one.

**Note on source spectrum**: The default source spectrum is, if *pysynphot* is installed, a G2V star spectrum from Castelli & Kurucz (2004). Without *pysynphot*, the default is a simple flat spectrum such that the same number of photons are detected at each wavelength.

</div>

In [ ]:
def create_psf_model(det='NRCA1', filt='F070W', fov=101, num=9, all_detectors=True, oversample=4,
                     add_distortion=True, detsampled=False, single_psf_centered=True, x0=None, y0=None):

    nrc = webbpsf.NIRCam()
    
    nrc.detector = det 
    nrc.filter = filt

    src = webbpsf.specFromSpectralType('G5V', catalog='phoenix')
    
    print("Using a {field}".format(field=fov), "px fov")
    
    psfs_dir = create_dir(fov=fov, num=num, distortion=add_distortion, detsampled=detsampled)
    
    if num > 1:
        
        if np.sqrt(num).is_integer():

            print("")
            print("Creating a grid of PSF for filter {filt}".format(filt=filt))
            print("")
            num = num
        
            if add_distortion:
                outname = 'PSF_{0}_fov{1}_npsfs{2}.fits'.format(filt, fov, num)
                
            else:
                outname = 'PSF_{0}_fov{1}_npsfs{2}.fits'.format(filt, fov, num)
                
        
            psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=src, all_detectors=all_detectors, 
                               fov_pixels=fov, add_distortion=True, save=True, 
                               outfile=os.path.join(psfs_dir,outname), use_detsampled_psf=detsampled)
            
        else:
            raise ValueError("You must choose a square number of cells to create (E.g. 9, 16, etc.)")
                        
    else:
        print("")
        print("Creating a single PSF for filter {filt}".format(filt=filt))
        print("")
        num = num
        
        if not single_psf_centered:
        
            x0 = x0
            y0 = y0
            nrc.detector_position = (x0,y0)
        
        if add_distortion:
            
            if single_psf_centered:
                
                outname = 'PSF_{0}_fov{1}_npsfs{2}_xdetcen_ydetcen.fits'.format(filt, fov, num)
                
            else:
                outname = 'PSF_{0}_fov{1}_npsfs{2}_x{3}_y{4}.fits'.format(filt, fov, num, x0, y0)
        
        else:

            
            if single_psf_centered:
                
                outname = 'PSF_{0}_fov{1}_npsfs{2}_xdetcen_ydetcen.fits'.format(filt, fov, num)
            else:
                outname = 'PSF_{0}_fov{1}_npsfs{2}_x{3}_y{4}.fits'.format(filt, fov, num, x0, y0)
    
        psf = nrc.psf_grid(num_psfs=num, oversample=oversample, source=src, all_detectors=all_detectors, 
                           fov_pixels=fov, add_distortion=True, save=True, outfile=os.path.join(psfs_dir,outname), 
                           use_detsampled_psf=detsampled, single_psf_centered=single_psf_centered)
        
    return psf     

In [ ]:
def create_dir(fov=11, num=1, oversample=4, distortion=True, detsampled=False):
    
    if distortion:
        
        if detsampled:
            
            psf_dir = 'PSF_MODELS/Distorted/Fov{}px_numPSFs{}_detsampled'.format(fov, num)
            
        else:
            
            psf_dir = 'PSF_MODELS/Distorted/Fov{}px_numPSFs{}_oversample{}'.format(fov, num, oversample)

    else:
        if detsampled:
            
            psf_dir = 'PSF_MODELS/Undistorted/Fov{}px_numPSFs{}_detsampled'.format(fov, num)
            
        else:
        
            psf_dir = 'PSF_MODELS/Undistorted/Fov{}px_numPSFs{}_oversample{}'.format(fov, num, oversample)
    
    if not os.path.exists(psf_dir):
        os.makedirs(psf_dir)
    return psf_dir

In [ ]:
filters = ['F070W', 'F090W', 'F115W', 'F140M', 'F150W2', 'F150W', 'F162M', 'F164N', 'F182M', 'F187N', 'F200W', 
           'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2', 'F323N', 'F335M', 'F356W', 'F360M', 'F405N', 
           'F410M', 'F430M', 'F444W', 'F460M', 'F466N', 'F470N', 'F480M']

### 3.1<font color='white'>-</font>Create PSFs for all detectors and filters<a class="anchor" id="psfs"></a> ###

In [ ]:
psf_list = []
    
for filt in filters:    
    psf_webbpsf = create_psf_model(filt=filt, fov=11, num=4, all_detectors=True, 
                                   add_distortion=True, detsampled=False)

    psf_list.append(psf_webbpsf)


### 3.2<font color='white'>-</font>Create PSFs for a subset of filters<a class="anchor" id="psf_filt"></a> ###

In [ ]:
psf_list = []
    
filters_subset = ['F115W', 'F164N', 'F200W', 'F210M', 'F277W', 'F335M', 'F444W', 'F466N']    

for filt in filters_subset:    
    psf_webbpsf = create_psf_model(filt=filt, fov=11, num=16, all_detectors=True, oversample=4,
                                   add_distortion=True, detsampled=False)
    
    psf_list.append(psf_webbpsf)


### 3.3<font color='white'>-</font>Create PSFs for a subset of detectors<a class="anchor" id="psf_det"></a> ###

In [ ]:
psf_list = []

filt = 'F200W'
det_subset = ['NRCB1']

for det in det_subset:
    
    # example with 1 PSF centered in the middle of the detector
    
    psf_webbpsf = create_psf_model(det=det, filt=filt, fov=41, num=1, all_detectors=False, oversample=4, 
                                   add_distortion=True, detsampled=False, single_psf_centered=True)

    psf_list.append(psf_webbpsf)


### 3.4<font color='white'>-</font>Create PSFs for a subset of detectors and filters<a class="anchor" id="psf_det_filt"></a> ###

In [ ]:
psf_list = []

filt_subset = ['F277W', 'F335M', 'F466N', 'F444W']
det_subset = ['NRCB5']

for det in det_subset:
    for filt in filt_subset:
        
        # example with a grid of 16 PSFs 
        
        psf_webbpsf = create_psf_model(det=det, filt=filt, fov=11, num=16, all_detectors=False, oversample=4, 
                                       add_distortion=True, detsampled=False)
        
        psf_list.append(psf_webbpsf)


### 3.5<font color='white'>-</font>Create PSFs for a subset of detectors and filters at specific positions in the detector<a class="anchor" id="psf_det_filt_pos"></a> ###

In [ ]:
psf_list = []

filt_subset = ['F115W']
det_subset = ['NRCB1']

positions = [(511, 511), (511, 1533), (1533, 511), (1533, 1533)]

for det in det_subset:
    
    for filt in filt_subset:
        
        for pos in positions:
            
            x0 = pos[0]
            y0 = pos[1]
    
            psf_webbpsf = create_psf_model(det=det, filt=filt, fov=41, num=1, all_detectors=False, oversample=4, 
                                   add_distortion=True, detsampled=False, single_psf_centered=False, x0=x0, y0=y0)

            psf_list.append(psf_webbpsf)
